# Use BoomSpikeSlab
## First check the result of varbvs on whole genome

In [1]:
res = readRDS("/home/min/GIT/cnv-gene-mapping/data/deletion_simu/deletion.genes.block30.binomial.whole.genome.prior.varbvs.nologodds.updateb0.rds")

In [2]:
mu0 = sum(res$b0 * res$w)

In [3]:
mu0

[1] 0.7770721

In [4]:
sd0 = sqrt(sum(res$sa * res$w))

In [5]:
sd0

[1] 0.8436502

In [6]:
pi0 = sum((10^(res$logodds) / (1 + 10^(res$logodds))) * res$w)

In [7]:
pi0

[1] 0.0437755

In [31]:
# library(varbvs)
# help(varbvs)

In [32]:
# varbvs::varbvs

In [8]:
library(data.table)
library(reticulate)

In [9]:
library(susieR)
library(BoomSpikeSlab)

Loading required package: Boom
Loading required package: MASS

Attaching package: ‘Boom’

The following object is masked from ‘package:stats’:

    rWishart


Attaching package: ‘BoomSpikeSlab’

The following object is masked from ‘package:stats’:

    knots



In [10]:
X = as.matrix(data.table::fread("/home/min/GIT/cnv-gene-mapping/data/deletion_simu/block_79_137/deletion.genes.block30.for_simu.sample.genes.block_79_137.gz", header = F))

In [12]:
y <- as.matrix(data.table::fread("/data/GIT/cnv-gene-mapping/data/deletion_simu/deletion.genes.block30.for_simu.sample.y"))

In [14]:
sampler = c(sample(1:sum(y), sum(y)*0.06), c((sum(y)+1):length(y)))
y = y[sampler, ]
X = X[sampler, ]

In [15]:
X_inter = cbind(1, X)

In [16]:
storage.mode(X) = 'double'
storage.mode(X_inter) = 'double'
storage.mode(y) = 'double'

In [17]:
dim(X_inter)

[1] 7108   60

In [18]:
N = rep(1, length(y))

In [19]:
X_center <- scale(X, scale = FALSE)

In [20]:
prior <- LogitZellnerPrior(X_inter, y, N, prior.success.probability = NULL, expected.model.size = 2, prior.information.weight = 0.05, 
                           diagonal.shrinkage = 0, optional.coefficient.estimate = NULL, prior.inclusion.probabilities = NULL)

In [21]:
names(prior)

[1] "prior.inclusion.probabilities" "mu"                           
[3] "sigma.guess"                   "prior.df"                     
[5] "sigma.upper.limit"             "max.flips"                    
[7] "siginv"                        "prior.success.probability"

In [78]:
prior$prior.success.probability

[1] 0.05655599

In [52]:
prior_glm <- SpikeSlabGlmPrior(X, weight = rep(1/nrow(X), nrow(X)), prior.inclusion.probabilities = rep(1/ncol(X), ncol(X)), expected.model.size = 1, 
                               optional.coefficient.estimate = rep(0, ncol(X)), mean.on.natural.scale = -1, max.flips = 5, diagonal.shrinkage = 0, prior.information.weight = 0.05)

In [53]:
names(prior_glm)

[1] "prior.inclusion.probabilities" "mu"                           
[3] "sigma.guess"                   "prior.df"                     
[5] "sigma.upper.limit"             "max.flips"                    
[7] "siginv"

In [42]:
prior1 <- prior

In [43]:
length(prior$mu); ncol(X)

[1] 60

[1] 59

In [63]:
prior1$prior.inclusion.probabilities <- c(prior$prior.inclusion.probabilities[1], rep(pi0, ncol(X)))
prior1$mu <- c(prior$mu[1], rep(0, 10), mu0, rep(0, 10), mu0, mu0, rep(0, 9), mu0, rep(0,10), mu0, rep(0,10), mu0, rep(0, 4))
# prior1$mu <- c(prior$mu[1], colSums(X_inter)[2:ncol(X_inter)]*0.001)
prior1$sigma.guess <- sd0

In [64]:
length(prior$mu) == ncol(X)

[1] FALSE

In [65]:
model1 <- logit.spike(y ~ X, niter = 1000, prior = prior1)

=-=-=-=-= Iteration 0 Thu Oct  3 22:48:23 2019 =-=-=-=-=
=-=-=-=-= Iteration 100 Thu Oct  3 22:48:53 2019 =-=-=-=-=
=-=-=-=-= Iteration 200 Thu Oct  3 22:49:17 2019 =-=-=-=-=
=-=-=-=-= Iteration 300 Thu Oct  3 22:49:46 2019 =-=-=-=-=
=-=-=-=-= Iteration 400 Thu Oct  3 22:50:16 2019 =-=-=-=-=
=-=-=-=-= Iteration 500 Thu Oct  3 22:50:45 2019 =-=-=-=-=
=-=-=-=-= Iteration 600 Thu Oct  3 22:51:12 2019 =-=-=-=-=
=-=-=-=-= Iteration 700 Thu Oct  3 22:51:41 2019 =-=-=-=-=
=-=-=-=-= Iteration 800 Thu Oct  3 22:52:09 2019 =-=-=-=-=
=-=-=-=-= Iteration 900 Thu Oct  3 22:52:33 2019 =-=-=-=-=


In [76]:
model0 <- logit.spike(y ~ X, niter = 30000)

=-=-=-=-= Iteration 0 Fri Oct  4 10:31:52 2019 =-=-=-=-=
=-=-=-=-= Iteration 3000 Fri Oct  4 10:44:58 2019 =-=-=-=-=
=-=-=-=-= Iteration 6000 Fri Oct  4 10:58:11 2019 =-=-=-=-=
=-=-=-=-= Iteration 9000 Fri Oct  4 11:11:12 2019 =-=-=-=-=
=-=-=-=-= Iteration 12000 Fri Oct  4 11:24:44 2019 =-=-=-=-=
=-=-=-=-= Iteration 15000 Fri Oct  4 11:37:53 2019 =-=-=-=-=
=-=-=-=-= Iteration 18000 Fri Oct  4 12:01:18 2019 =-=-=-=-=
=-=-=-=-= Iteration 21000 Fri Oct  4 12:17:17 2019 =-=-=-=-=
=-=-=-=-= Iteration 24000 Fri Oct  4 12:30:34 2019 =-=-=-=-=
=-=-=-=-= Iteration 27000 Fri Oct  4 12:48:20 2019 =-=-=-=-=


In [77]:
summary(model0)

null log likelihood:            -1545.166 
posterior mean log likelihood:  -1545.593 
posterior max log likelihood:   -1543.954 
mean deviance R-sq:             -0.0002758504 

predicted vs observed success rates, by decile:
                     predicted   observed
(0.050313,0.056566] 0.05656074 0.05632411
(0.056566,0.056569] 0.05656864 0.17647059

summary of coefficients:
                 mean       sd  mean.inc   sd.inc inc.prob
(Intercept) -2.82e+00 5.20e-02    -2.820    0.052 1.00e+00
XV7         -1.79e+01 1.74e+02  -979.000  844.000 1.83e-02
XV51        -7.26e+00 7.31e+01  -404.000  371.000 1.80e-02
XV11        -1.67e+01 1.67e+02  -995.000  833.000 1.68e-02
XV52        -1.02e+01 1.02e+02  -678.000  485.000 1.50e-02
XV9         -1.57e+01 2.17e+02 -1150.000 1470.000 1.37e-02
XV53        -1.73e+01 1.78e+02 -1470.000  748.000 1.18e-02
XV12        -1.06e+01 1.69e+02  -934.000 1290.000 1.13e-02
XV10        -7.42e+00 9.21e+01  -698.000  562.000 1.06e-02
XV8         -1.14e+01 1.65e+02 -1

In [74]:
model <- logit.spike(y ~ X, niter = 30000, prior = prior)
# model <- logit.spike(y ~ X_inter, niter = 100)

=-=-=-=-= Iteration 0 Thu Oct  3 23:29:17 2019 =-=-=-=-=
=-=-=-=-= Iteration 3000 Thu Oct  3 23:44:03 2019 =-=-=-=-=
=-=-=-=-= Iteration 6000 Fri Oct  4 00:00:58 2019 =-=-=-=-=
=-=-=-=-= Iteration 9000 Fri Oct  4 00:15:56 2019 =-=-=-=-=
=-=-=-=-= Iteration 12000 Fri Oct  4 00:34:17 2019 =-=-=-=-=
=-=-=-=-= Iteration 15000 Fri Oct  4 00:47:14 2019 =-=-=-=-=
=-=-=-=-= Iteration 18000 Fri Oct  4 01:04:03 2019 =-=-=-=-=
=-=-=-=-= Iteration 21000 Fri Oct  4 01:18:54 2019 =-=-=-=-=
=-=-=-=-= Iteration 24000 Fri Oct  4 01:34:07 2019 =-=-=-=-=
=-=-=-=-= Iteration 27000 Fri Oct  4 01:53:01 2019 =-=-=-=-=


In [75]:
summary(model) # prior 1000

null log likelihood:            -1545.166 
posterior mean log likelihood:  -1545.555 
posterior max log likelihood:   -1544.175 
mean deviance R-sq:             -0.0002518388 

predicted vs observed success rates, by decile:
                 predicted   observed
(0.0429,0.0566] 0.05661999 0.05611166
(0.0566,0.0567] 0.05664569 0.36363636

summary of coefficients:
                 mean       sd  mean.inc   sd.inc inc.prob
(Intercept) -2.81e+00 5.13e-02   -2.8100 5.13e-02 1.00e+00
XV2         -3.71e+01 1.89e+02 -671.0000 4.70e+02 5.53e-02
XV7         -2.88e+01 1.48e+02 -534.0000 3.72e+02 5.40e-02
XV56        -2.55e+01 1.43e+02 -552.0000 3.91e+02 4.63e-02
XV10        -2.91e+01 1.75e+02 -694.0000 5.22e+02 4.20e-02
XV3         -1.67e+01 9.96e+01 -431.0000 2.78e+02 3.88e-02
XV11        -1.94e+01 1.24e+02 -504.0000 3.94e+02 3.85e-02
XV13        -9.79e+00 1.02e+02 -720.0000 5.04e+02 1.36e-02
XV53        -4.28e+00 5.51e+01 -493.0000 3.31e+02 8.67e-03
XV47        -1.96e+00 2.92e+01 -296.0000 2.05

In [66]:
summary(model1) # prior1 1000

null log likelihood:            -1545.166 
posterior mean log likelihood:  -1545.637 
posterior max log likelihood:   -1544.987 
mean deviance R-sq:             -0.0003043409 

predicted vs observed success rates, by decile:
                 predicted   observed
(0.0564,0.0582] 0.05739954 0.05128205

summary of coefficients:
                 mean     sd mean.inc  sd.inc inc.prob
(Intercept) -2.820000 0.0478  -2.8200 0.04780    1.000
XV22         0.000816 0.1230   0.0742 1.23000    0.011
XV55         0.002830 0.0449   0.7070 0.08550    0.004
XV44         0.001430 0.0243   0.3580 0.16100    0.004
XV33         0.003210 0.0506   0.8010 0.00488    0.004
XV23         0.003010 0.0477   0.7540 0.04700    0.004
XV11         0.004150 0.0664   1.0400 0.20300    0.004
XV5         -0.000840 0.0266  -0.8400 0.00000    0.001
XV59         0.000000 0.0000   0.0000 0.00000    0.000
XV58         0.000000 0.0000   0.0000 0.00000    0.000
XV57         0.000000 0.0000   0.0000 0.00000    0.000
XV56         

In [19]:
n <- 100
p <- 10
ngood <- 3
niter <- 1000
x <- cbind(1, matrix(rnorm(n * (p-1)), nrow=n))
beta <- c(rnorm(ngood), rep(0, p - ngood))
prob <- plogis(x %*% beta)
y <- runif(n) < prob
x <- x[,-1]
model <- logit.spike(y ~ x, niter=niter)

=-=-=-=-= Iteration 0 Wed Oct  2 22:00:39 2019 =-=-=-=-=
=-=-=-=-= Iteration 100 Wed Oct  2 22:00:39 2019 =-=-=-=-=
=-=-=-=-= Iteration 200 Wed Oct  2 22:00:39 2019 =-=-=-=-=
=-=-=-=-= Iteration 300 Wed Oct  2 22:00:39 2019 =-=-=-=-=
=-=-=-=-= Iteration 400 Wed Oct  2 22:00:39 2019 =-=-=-=-=
=-=-=-=-= Iteration 500 Wed Oct  2 22:00:40 2019 =-=-=-=-=
=-=-=-=-= Iteration 600 Wed Oct  2 22:00:40 2019 =-=-=-=-=
=-=-=-=-= Iteration 700 Wed Oct  2 22:00:40 2019 =-=-=-=-=
=-=-=-=-= Iteration 800 Wed Oct  2 22:00:40 2019 =-=-=-=-=
=-=-=-=-= Iteration 900 Wed Oct  2 22:00:40 2019 =-=-=-=-=


In [20]:
summary(test)

null log likelihood:            -57.30569 
posterior mean log likelihood:  -69.21457 
posterior max log likelihood:   -67.30182 
mean deviance R-sq:             -0.2078131 

predicted vs observed success rates, by decile:
              predicted observed
(0.49,0.494]  0.4923392      0.0
(0.494,0.496] 0.4952206      0.1
(0.496,0.498] 0.4969029      0.2
(0.498,0.499] 0.4983347      0.2
(0.499,0.5]   0.4996715      0.4
(0.5,0.502]   0.5011072      0.3
(0.502,0.503] 0.5022994      0.0
(0.503,0.504] 0.5036059      0.4
(0.504,0.507] 0.5056010      0.3
(0.507,0.512] 0.5090135      0.7

summary of coefficients:
        mean     sd mean.inc sd.inc inc.prob
x5 -0.017700 0.0942   -0.455 0.1720    0.039
x1 -0.011400 0.0738   -0.394 0.1940    0.029
x9 -0.000636 0.0142   -0.318 0.0138    0.002
x2 -0.001020 0.0228   -0.510 0.0000    0.002
x8  0.000000 0.0000    0.000 0.0000    0.000
x7  0.000000 0.0000    0.000 0.0000    0.000
x6  0.000000 0.0000    0.000 0.0000    0.000
x4  0.000000 0.0000    0.000 

In [36]:
summary(test)

null log likelihood:            -67.68585 
posterior mean log likelihood:  -69.297 
posterior max log likelihood:   -67.94729 
mean deviance R-sq:             -0.02380324 

predicted vs observed success rates, by decile:
                predicted  observed
(0.4974,0.4986] 0.4982661 0.6666667
(0.4986,0.499]  0.4988171 0.3000000
(0.499,0.4995]  0.4991941 0.6000000
(0.4995,0.4999] 0.4996560 0.5000000
(0.4999,0.5002] 0.5000711 0.4000000
(0.5002,0.5004] 0.5002891 0.8000000
(0.5004,0.5007] 0.5005541 0.8000000
(0.5007,0.501]  0.5009263 0.4000000
(0.501,0.5014]  0.5011724 0.6000000
(0.5014,0.5027] 0.5018429 0.9000000

summary of coefficients:
                 mean      sd mean.inc  sd.inc inc.prob
x3           0.004290 0.04540   0.4290 0.16300    0.010
x1           0.001960 0.02430   0.2450 0.12700    0.008
x9          -0.000974 0.01540  -0.2440 0.00585    0.004
x2          -0.000161 0.00413  -0.0536 0.06500    0.003
x7          -0.000214 0.00483  -0.1070 0.01970    0.002
x5           0.000281

In [39]:
colSums(test$beta)

(Intercept)          x1          x2          x3          x4          x5 
  0.0000000   1.9614600  -0.1608144   4.2932854   0.0000000   0.2814675 
         x6          x7          x8          x9 
  0.0000000  -0.2143147   0.0000000  -0.9741396

In [10]:
dim(model$beta)

[1] 1000   60

In [98]:
mean(model$beta[, 20])

[1] 0.009948308

In [13]:
mean(model$beta[, 50])

[1] 0.001787244

In [45]:
res1 = susie(X, y, L = 3, scaled_prior_variance = 0.005, estimate_prior_method = "optim")

In [47]:
max(res1$pip)

[1] 0

In [29]:
res2 = readRDS("/home/min/GIT/cnv-gene-mapping/data/deletion_simu/block_79_137/deletion.genes.block30.for_simu.sample.genes.block_79_137.varbvs.rds")

In [30]:
names(res2)

[1] "fit"  "mu"   "beta"

In [48]:
max(res2$fit$pip)

[1] 0.02698148

**Make input prior as the format of output of SpikeSlabPrior**

pi: vector of prior.inclusion.probabilities

b: optional.coefficient.estimate

    spike.slab.prior Create a spike and slab prior for use with lm.spike.
    SpikeSlabPrior(x,
    y = NULL,
    expected.r2 = .5,
    prior.df = .01,
    expected.model.size = 1,
    prior.information.weight = .01,
    diagonal.shrinkage = .5,
    optional.coefficient.estimate = NULL,
    max.flips = -1,
    mean.y = mean(y, na.rm = TRUE),
    sdy = sd(as.numeric(y), na.rm = TRUE),
    prior.inclusion.probabilities = NULL,
    sigma.upper.limit = Inf)
    
    This object is intended for use with logit.spike.
    LogitZellnerPrior(
    predictors,
    successes = NULL,
    trials = NULL,
    prior.success.probability = NULL,
    expected.model.size = 1,
    prior.information.weight = .01,
    diagonal.shrinkage = .5,
    optional.coefficient.estimate = NULL,
    max.flips = -1,
    prior.inclusion.probabilities = NULL)

    logit.spike    Spike and slab logistic regression
    logit.spike(formula,
    niter,
    data,
    subset,
    prior = NULL,
    na.action = options("na.action"),
    contrasts = NULL,
    drop.unused.levels = TRUE,
    initial.value = NULL,
    ping = niter / 10,
    nthreads = 0,
    clt.threshold = 2,
    mh.chunk.size = 10,
    proposal.df = 3,
    sampler.weights = c("DA" = .333, "RWM" = .333, "TIM" = .333),
    seed = NULL,
    ...)
    
    lm.spike       Spike and slab regression
    lm.spike(formula,
    niter,
    data,
    subset,
    prior = NULL,
    error.distribution = c("gaussian", "student"),
    contrasts = NULL,
    drop.unused.levels = TRUE,
    model.options = SsvsOptions(),
    ping = niter / 10,
    seed = NULL,
    ...)